In [ ]:
!pip install tensorflow-gpu
!pip install unidecode
!pip install inflect
!pip install matplotlib
!pip install phonemizer
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/s

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import phonemizer
import num2words
import torch
df = pd.read_csv('/content/drive/MyDrive/bn_bd/line_index.tsv', delimiter='\t', header=None, names=['audio_id', 'bangla_text'])

In [ ]:
df

,audio_id,bangla_text
0,ban_00737_00012222450,এইচআর টেক্সটাইল বাংলাদেশের ভেতরে একাধিক আউটলেট...
1,ban_00737_00015581920,স্ট্যান্ডার্ড ব্যাংক এ ইসলামী ব্যাংকিং এর সুবি...
2,ban_00737_00028634754,লাফার্জ সুরমা সিমেন্ট সর্বাধিক ব্যবহৃত সিমেন্ট...
3,ban_00737_00035050432,পিপলস ইন্স্যুরেন্স অব চায়না ছেষট্টি বছর আগে ব্...
4,ban_00737_00068052117,বয়গেস একটি ইন্ডাস্ট্রিয়াল গ্রুপ
...,...,...
1886,ban_03042_02119461967,ঢাকা ময়মনসিংহ মহাসড়ক এ থাকতে অল্প ডান দিক ধরুন
1887,ban_03042_02121456548,দা লর্ড অফ দা রিংস বইয়ে যা রয়েছ
1888,ban_03042_02132375596,বাড়ি।
1889,ban_03042_02141733462,রবিবারে সাত নভেম্বর দুই হাজার বারো সাল।


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import codecs
import unicodedata
import numpy as np
import pandas as pd

from torch.utils.data import Dataset

In [ ]:
#vocab = "PE abcdefghijklmnopqrstuvwxyz'.?"  # P: Padding, E: EOS.
#vocab = "PE &্0123456789অoোyয়ওঐৌৈঔüuুwউঊূaাআàâeèéেêএিইঈীiংঙঞঃশষসsহhণনnটtঠডdঢৎতথদধপpফfবbভvমmলlযঝজjzছচcxকkqখগgঘঋৃড়rর"
vocab = "PE অআইঈউঊঋএঐওঔা ি ী ু ূ ৃ ে ৈ ো ৌক খ গ ঘ ঙ চ ছ জ ঝ ঞ ট ঠ ড ঢ ণত থ দ ধ ন প ফ ব ভমযরলশষসহড়ঢ়য়ৎংঃঁ্ঽ‍্য‍  ‍্র'"
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = {idx: char for idx, char in enumerate(vocab)}

In [ ]:
def text_normalize(text):
    text = ''.join(char for char in unicodedata.normalize('NFKC', text)
                   if unicodedata.category(char) != 'Mn')  # Strip accents
    print(text)
    #text = text.lower()
    text = re.sub("[^{}]()".format(vocab), " ", text)
    text = re.sub("[ ()]+", " ", text)
    return text

In [ ]:
ss = text_normalize("আমার,++ নাম :(শামীম) ক্কা? কক্ষ মুখ্য পত্র, বন্ধু")
print(unicodedata.normalize('NFKC', "আমার,++ নাম :(শামীম) ক্কা? কক্ষ মুখ্য পত্র, বন্ধু"))
#print(ss)

আমার,++ নাম :(শামীম) ককা? ককষ মখয পতর, বনধ
আমার,++ নাম :(শামীম) ক্কা? কক্ষ মুখ্য পত্র, বন্ধু


In [ ]:
pwd


'/content'

In [ ]:
cd '/content/drive/MyDrive/TTSBn/datasets/bn_bd'

/content/drive/MyDrive/TTSBn/datasets/bn_bd


In [ ]:
ls

line_index.tsv  mags/  mels/  wavs/


In [ ]:
lines = codecs.open('line_index.tsv', 'r', 'utf-8').readlines()
len(lines[1])
#np.shape(lines)
line = lines[3]
print(line)

ban_00737_00035050432	পিপলস ইন্স্যুরেন্স অব চায়না ছেষট্টি বছর আগে ব্যবসা চালু করে



In [ ]:
fname, text = line.strip().split("\t")
print(fname, "\n", text)

ban_00737_00035050432 
 পিপলস ইন্স্যুরেন্স অব চায়না ছেষট্টি বছর আগে ব্যবসা চালু করে


In [ ]:
cd '/content/drive/MyDrive/TTSBn'

/content/drive/MyDrive/TTSBn


In [ ]:
pwd

'/content/drive/MyDrive/TTSBn'

In [ ]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.3 MB/s eta 0:00:00


In [ ]:
import os
import sys
import torch


from os.path import exists, join, expanduser
from audio import preprocess
from utils import download_file
from datasets.lj_speech import LJSpeech

In [ ]:
cd '/content/drive/MyDrive/TTSBn/datasets'

/content/drive/MyDrive/TTSBn/datasets


In [ ]:
datasets_path = os.path.join('/content/drive/MyDrive/TTSBn', 'datasets')
dataset_path = os.path.join(datasets_path, 'bn_bd')

if os.path.isdir(dataset_path) and False:
  print("bn_bd dataset folder already exists")
  #sys.exit(0)

In [ ]:
pwd

'/content/drive/MyDrive/TTSBn/datasets'

In [ ]:
cd '/content/drive/MyDrive/TTSBn/datasets/bn_bd'

/content/drive/MyDrive/TTSBn/datasets/bn_bd


In [ ]:
!pip uninstall librosa

Found existing installation: librosa 0.5.1
Uninstalling librosa-0.5.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/librosa-0.5.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/librosa/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/audio._zc_wrapper-1161.py310.1.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/audio._zc_wrapper-1161.py310.2.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/audio._zc_wrapper-1161.py310.nbi
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-audio._zc_wrapper-1161.py310.1.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-audio._zc_wrapper-1161.py310.2.nbc
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-audio._zc_wrapper-1161.py310.nbi
    /usr/local/lib/python3.10/dist-packages/librosa/core/__pycache__/guf-pitch._pi_wrapper-445.py310.1.nbc
    

In [ ]:
!pip install --upgrade pysoundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dataset_path = '/content/drive/MyDrive/TTSBn/datasets/bn_bd'

In [ ]:
print("pre processing...")
lj_speech = LJSpeech([])
print(lj_speech)
preprocess(dataset_path, lj_speech)

pre processing...


100%|██████████| 1891/1891 [1:25:45<00:00,  2.72s/it]


In [ ]:
cd '/content/drive/My Drive/TTSBn/'

/content/drive/My Drive/TTSBn


In [ ]:
!pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorboardX import SummaryWriter

In [ ]:
!python train-text2mel.py --dataset=ljspeech

use_gpu True
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
epoch   0 with lr=1.25e-06
100% 1856/1856 [03:17<00:00,  9.41audios/s, l1=0.40553, att=0.00098]
train epoch loss 0.406507, step=29, total time elapsed: 0h 3m 17s 
epoch   0 with lr=3.75e-05
100% 64/64 [00:02<00:00, 27.87audios/s]
valid epoch loss 0.365677
epoch   1 with lr=3.75e-05
100% 1856/1856 [01:00<00:00, 30.81audios/s, l1=0.38792, att=0.00059]
train epoch loss 0.388506, step=58, total time elapsed: 0h 4m 20s 
epoch   1 with lr=7.38e-05
100% 64/64 [00:00<00:00, 72.84audios/s]
valid epoch loss 0.360903


In [ ]:
import sys
import time
import argparse
from tqdm import *

import torch
import torch.nn.functional as F

# project imports
from models import SSRN
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import SSRNDataLoader

In [ ]:
!python train-ssrn.py --dataset=ljspeech

use_gpu True
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
epoch   0 with lr=5.00e-07
100% 1872/1872 [07:41<00:00,  4.05audios/s, l1=0.24350]
train epoch loss 0.243504, step=78, total time elapsed: 0h 7m 42s 
epoch   0 with lr=3.95e-05
100% 24/24 [00:04<00:00,  5.07audios/s]
valid epoch loss 0.057785
epoch   1 with lr=3.95e-05
100% 1872/1872 [04:22<00:00,  7.13audios/s, l1=0.07879]
train epoch loss 0.078792, step=156, total time elapsed: 0h 12m 9s 
epoch   1 with lr=7.85e-05
100% 24/24 [00:01<00:00, 15.77audios/s]
valid epoch loss 0.044318
epoch   2 with lr=7.85e-0

In [ ]:
import os
import sys
import argparse
from tqdm import *

import IPython
from IPython.display import Audio


import numpy as np
import torch

from models import Text2Mel
from models import SSRN
from hparams import HParams as hp
from audio import save_to_wav
from utils import get_last_checkpoint_file_name, load_checkpoint, save_to_png
from datasets.lj_speech import vocab, get_test_data, text_normalize
#from datasets.lj_speech import vocab, get_test_data



In [ ]:
#!python synthesize.py --dataset=ljspeech

## Testing the bangla text to speech using train model

In [ ]:
torch.set_grad_enabled(False)
text2mel = Text2Mel(vocab)
#print(text2mel)
text2mel.load_state_dict(torch.load("/content/drive/MyDrive/TTSBn/logdir/ljspeech-text2mel/step-001K.pth")['state_dict'])
text2mel = text2mel.eval()
ssrn = SSRN()
ssrn.load_state_dict(torch.load("/content/drive/MyDrive/TTSBn/logdir/ljspeech-ssrn/step-000K.pth")['state_dict'])
ssrn = ssrn.eval()

In [ ]:

SENTENCES = [
        "স্ট্যান্ডার্ড ব্যাংক এ ইসলামী ব্যাংকিং এর সুবিধা রয়েছে",
        "লাফার্জ সুরমা সিমেন্ট সর্বাধিক ব্যবহৃত সিমেন্ট উৎপাদন করে",
        "বয়গেস একটি ইন্ডাস্ট্রিয়াল গ্রুপ",

    ]


In [ ]:
# synthetize by one by one because there is a batch processing bug!
for i in range(len(SENTENCES)):
    sentence = SENTENCES[i]
    normalized_sentence = "".join([c if c.lower() in vocab else '' for c in sentence])
    print(normalized_sentence)

    #nm_sen = text_normalize(sentence)
    #normalized_sentence = nm_sen
    #print(nm_sen)

    #normalized_sentence = sentence
    #print(normalized_sentence)


    sentences = [normalized_sentence]
    max_N = len(normalized_sentence)+3
    L = torch.from_numpy(get_test_data(sentences, max_N))
    zeros = torch.from_numpy(np.zeros((1, hp.n_mels, 1), np.float32))
    Y = zeros
    A = None

    for t in range(hp.max_T):
      _, Y_t, A = text2mel(L, Y, monotonic_attention=True)
      Y = torch.cat((zeros, Y_t), -1)
      _, attention = torch.max(A[0, :, -1], 0)
      attention = attention.item()
      if L[0, attention] == vocab.index('E'):  # EOS
          break

    _, Z = ssrn(Y)

    Z = Z.cpu().detach().numpy()
    save_to_wav(Z[0, :, :].T, '%d.wav' % (i + 1))
    IPython.display.display(Audio('%d.wav' % (i + 1), rate=hp.sr))

স্ট্যান্ডার্ড ব্যাংক এ ইসলামী ব্যাংকিং এর সুবিধা রয়েছে


লাফার্জ সুরমা সিমেন্ট সর্বাধিক ব্যবহৃত সিমেন্ট উৎপাদন করে


বয়গেস একটি ইন্ডাস্ট্রিয়াল গ্রুপ


In [ ]:
#!pip install librosa

In [ ]:
#!pip show librosa